<a href="https://colab.research.google.com/github/nrocka/Anime-SD-Prompt-to-Danbooru/blob/main/Anime_SD_Prompt_to_Danbooru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I am not a programmer at all, so im lacking a lot of knowledge. This thing was built entirely using a Chatbot AI. You will probably notice. The cells do work though (as far as i was able to test)

## 🟥🟥🟥 **Preperation** 🟥🟥🟥

In [ ]:
# Import Cell
# @title 0️⃣ Import Dependencies
# @markdown This cell contains all necessary imports.
import re
import requests
import pandas as pd
from IPython.display import display, HTML, Javascript

In [165]:
# Global Parameters Cell
# @title ✨ Global Parameters used throughout the notebook
# @markdown This parameter defines the maximum number of characters allowed before inserting a newline, making the output more readable on different window sizes. (Will not affect "Copy to Clipboard")
newline_threshold = 240 # @param {type:"integer"}

# @markdown This parameter decides whether a newline should cut off only after completing a tag or directly after reaching the threshold. (Will not affect "Copy to Clipboard")
only_cut_after_complete_tags = True # @param {type:"boolean"}

## 🟩🟩🟩 **Prompt Editor** 🟩🟩🟩

In [184]:
# First Cell: Clean Up Prompt
# @title 1️⃣ Clean Up Prompt
# @markdown This cell removes text inside chevrons (like <LoRA:_> triggers) and removes weights, including their brackets.

input_prompt = ""  # @param {type:"string"}

def clean_up_prompt(input_prompt):
    # 1. Remove chevrons (< and >) and all text inside them
    cleaned_prompt = re.sub(r'<.*?>', '', input_prompt)

    # 2. Remove weights from tags in brackets, e.g., (collarbone:1.4) -> (collarbone)
    cleaned_prompt = re.sub(r'\(([^:]*):[^\)]*\)', r'(\1)', cleaned_prompt)

    # 3. Remove brackets unless preceded by a backslash (\)
    cleaned_prompt = re.sub(r'(?<!\\)\(', '', cleaned_prompt)
    cleaned_prompt = re.sub(r'(?<!\\)\)', '', cleaned_prompt)

    # 4. Remove any double spaces between tags
    cleaned_prompt = re.sub(r'\s{2,}', ' ', cleaned_prompt)

    return cleaned_prompt

# Clean the input prompt
cleaned_prompt = clean_up_prompt(input_prompt)
print("Cleaned Prompt:")

if only_cut_after_complete_tags:
    output_lines = []
    current_line = ""
    for tag in cleaned_prompt.split(', '):
        if len(current_line) + len(tag) + 2 <= newline_threshold:
            current_line += (', ' if current_line else '') + tag
        else:
            current_line = current_line.rstrip(', ') + ','  # Ensure last tag in the line ends with a comma
            output_lines.append(current_line)
            current_line = tag
    if current_line:
        current_line = current_line.rstrip(', ')  # Remove trailing commas after processing all lines
        output_lines.append(current_line)
    cleaned_output = '\n'.join(output_lines)
else:
    cleaned_output = '\n'.join([cleaned_prompt[i:i + newline_threshold] for i in range(0, len(cleaned_prompt), newline_threshold)])

print(cleaned_output)

# Display a copy-to-clipboard button (for Jupyter Notebooks or compatible environments)
from IPython.core.display import HTML
display(HTML(f"""
    <button onclick="navigator.clipboard.writeText('{cleaned_output}')">Copy to Clipboard</button>
"""))


Cleaned Prompt:







---




In [185]:
# Second Cell: Remove Quality, Source and Score Tags
# @title 2️⃣ Remove Quality, Source, and Score Tags
# @markdown This cell removes score_, source_, quality tags, and date tags.

def remove_quality_source_tags(cleaned_prompt):
    # 1. Remove all score_ and source_ prompts
    cleaned_prompt = re.sub(r'\b(score_|source_)[^,\s]*[\s,]*', '', cleaned_prompt)

    # 2. Remove all quality tags used by danbooru
    quality_tags = ["masterpiece", "best quality"]
    for tag in quality_tags:
        cleaned_prompt = cleaned_prompt.replace(tag, '')

    # 3. Remove all date tags like Year Range
    date_tags = ["old", "early", "mid", "recent", "newest"]
    for tag in date_tags:
        cleaned_prompt = cleaned_prompt.replace(tag, '')

    # Clean up extra spaces and commas left behind
    cleaned_prompt = re.sub(r',\s*', ', ', cleaned_prompt).strip(', ')  # Remove trailing commas
    cleaned_prompt = re.sub(r'\s+', ' ', cleaned_prompt).strip()  # Remove excess spaces

    return cleaned_prompt

cleaned_prompt = remove_quality_source_tags(cleaned_prompt)
if only_cut_after_complete_tags:
    output_lines = []
    current_line = ""
for tag in cleaned_prompt.split(', '):
    if len(current_line) + len(tag) + 2 <= newline_threshold:
        current_line += (', ' if current_line else '') + tag
    else:
        current_line = current_line.rstrip(', ') + ','  # Make sure last tag in line ends with a comma
        output_lines.append(current_line)
        current_line = tag
if current_line:
    current_line = current_line.rstrip(', ')  # Remove any trailing commas after processing all lines
    output_lines.append(current_line)
    cleaned_output = '\n'.join(output_lines)
else:
    cleaned_output = '\n'.join([cleaned_prompt[i:i + newline_threshold] for i in range(0, len(cleaned_prompt), newline_threshold)])

print("Cleaned Prompt:")
print(cleaned_output)

display(HTML(f"""
    <button onclick="navigator.clipboard.writeText('{cleaned_prompt}')">Copy to Clipboard</button>
"""))

Cleaned Prompt:





---



In [186]:
# Third Cell: Format to Danbooru Input
# @title 3️⃣ Format to Danbooru Input
# @markdown This cell replaces spaces in tags with underscores, removes commas, and removes backslashes (\).

def format_to_danbooru_input(cleaned_prompt):
    # 1. Replace the spaces in every individual tag with underscores (_)
    tags = cleaned_prompt.split(', ')
    tags = [tag.strip().replace(' ', '_') for tag in tags]

    # 2. Remove every backslash
    tags = [tag.replace('\\', '') for tag in tags]

    # 3. Remove every comma
    formatted_prompt = ' '.join(tags)

    return formatted_prompt

# Format the cleaned prompt for Danbooru
formatted_prompt = format_to_danbooru_input(cleaned_prompt)
print("Formatted Prompt for Danbooru:")

if only_cut_after_complete_tags:
    output_lines = []
    current_line = ""
    for tag in formatted_prompt.split(' '):
        if len(current_line) + len(tag) + 1 <= newline_threshold:  # Use 1 instead of 2 since there's no comma
            current_line += (' ' if current_line else '') + tag
        else:
            output_lines.append(current_line)
            current_line = tag
    if current_line:
        output_lines.append(current_line)
    formatted_output = '\n'.join(output_lines)
else:
    formatted_output = '\n'.join([formatted_prompt[i:i + newline_threshold] for i in range(0, len(formatted_prompt), newline_threshold)])

print(formatted_output)

# Display a copy-to-clipboard button (for Jupyter Notebooks or compatible environments)
from IPython.core.display import HTML
display(HTML(f"""
    <button onclick="navigator.clipboard.writeText('{formatted_output}')">Copy to Clipboard</button>
"""))


Formatted Prompt for Danbooru:





---



In [187]:
# Fourth Cell: List Unsupported Tags
# @title 4️⃣ List unsupported and unknown Tags
# @markdown This cell lists unsupported or deprecated tags based on a CSV file containing selected tags. Chose the output:

output_option = "C - Both A and B" # @param ["A - Only return unsupported and unknown tags", "B - Return set of tags without unsupported and unknown tags", "C - Both A and B"]

def check_tags_against_csv(formatted_prompt, option):
    # Load selected tags from CSV
    # @markdown Chose a custom CSV to compare your tags to. A Tag will be listed as unsupported if it can't be found inside this CSV.
    # Default = https://huggingface.co/SmilingWolf/wd-vit-large-tagger-v3/raw/main/selected_tags.csv
    csv_url = "https://huggingface.co/SmilingWolf/wd-vit-large-tagger-v3/raw/main/selected_tags.csv" # @param {type:"string"}
    url = csv_url
    selected_tags = pd.read_csv(url)['name'].tolist()

    # Split tags from formatted prompt
    tags = formatted_prompt.split(' ')

    unsupported_tags = []
    supported_tags = []

    # Check each tag against the list of selected tags
    for tag in tags:
        if tag not in selected_tags:
            unsupported_tags.append(tag)
        else:
            supported_tags.append(tag)

    if option == "A - Only return unsupported and unknown tags" or option == "C - Both A and B":
        # Only return unsupported and unknown tags
        if unsupported_tags:
            unsupported_output = "Unsupported Tags: " + ' '.join(unsupported_tags)

            if only_cut_after_complete_tags:
                output_lines = []
                current_line = ""
                for tag in unsupported_tags:
                    if len(current_line) + len(tag) + 2 <= newline_threshold:
                        current_line += (' ' if current_line else '') + tag
                    else:
                        output_lines.append(current_line)
                        current_line = tag
                if current_line:
                    output_lines.append(current_line)
                unsupported_output = '\n'.join(output_lines)
            else:
                unsupported_output = '\n'.join([unsupported_output[i:i + newline_threshold] for i in range(0, len(unsupported_output), newline_threshold)])

            print("Unsupported Tags:")
            print(unsupported_output)
            display(HTML(f"""
                <button onclick="navigator.clipboard.writeText('{ ' '.join(unsupported_tags) }')">Copy to Clipboard</button>
            """))
        else:
            print("No unsupported tags found.")

    if option == "B - Return set of tags without unsupported and unknown tags" or option == "C - Both A and B":
        # Return set of tags without unsupported and unknown tags
        if supported_tags:
            supported_output = "Supported Tags: " + ' '.join(supported_tags)

            if only_cut_after_complete_tags:
                output_lines = []
                current_line = ""
                for tag in supported_tags:
                    if len(current_line) + len(tag) + 2 <= newline_threshold:
                        current_line += (' ' if current_line else '') + tag
                    else:
                        output_lines.append(current_line)
                        current_line = tag
                if current_line:
                    output_lines.append(current_line)
                supported_output = '\n'.join(output_lines)
            else:
                supported_output = '\n'.join([supported_output[i:i + newline_threshold] for i in range(0, len(supported_output), newline_threshold)])

            print("Supported Tags:")
            print(supported_output)
            display(HTML(f"""
                <button onclick="navigator.clipboard.writeText('{ ' '.join(supported_tags) }')">Copy to Clipboard</button>
            """))

# Use the formatted prompt from the previous cell
check_tags_against_csv(formatted_prompt, output_option)


Unsupported Tags:





---



## ✨✨✨ **Extras & Curate Tags** ✨✨✨

In [196]:
# Add Tags to Danbooru Tags Cell
# @title ➕ Add Tags to Danbooru Tags
# @markdown This cell accepts a Danbooru tag list from the user and allows them to add additional tags.

# @markdown Check this box to reuse the result of this cell for the next run. It will not add it to the input_tags visually though, but will do it in the background.
reuse_result = False # @param {type:"boolean"}

# @markdown Enter your Danbooru tag list here. Use to copy to clipboard from above cells if you want to use their outputs.
if not reuse_result:
    input_tags = "" # @param {type:"string"}

# @markdown Enter additional tags to add to end of the existing tag list:
additional_tags = "" # @param {type:"string"}

def add_tags(input_tags, additional_tags):
    # Split input tags and additional tags into lists
    base_tags = input_tags.split(' ')
    tags_to_add = additional_tags.split(' ') if additional_tags else []

    # Combine and remove duplicates, keeping order
    final_tags = base_tags + [tag for tag in tags_to_add if tag not in base_tags]

    # Create the final prompt
    final_prompt = ' '.join(final_tags)
    return final_prompt

# Get the final prompt based on user input
if reuse_result:
    input_tags = final_prompt
final_prompt = add_tags(input_tags, additional_tags)

# Update input_tags to include the additional tags
input_tags = final_prompt

print("Final Prompt After Adding Tags:")
if only_cut_after_complete_tags:
    output_lines = []
    current_line = ""
    for tag in final_prompt.split(' '):
        if len(current_line) + len(tag) + 1 <= newline_threshold:
            current_line += (' ' if current_line else '') + tag
        else:
            output_lines.append(current_line)
            current_line = tag
    if current_line:
        output_lines.append(current_line)
    cleaned_output = '\n'.join(output_lines)
else:
    cleaned_output = '\n'.join([final_prompt[i:i + newline_threshold] for i in range(0, len(final_prompt), newline_threshold)])

print(cleaned_output)

display(HTML(f"""
    <button onclick="navigator.clipboard.writeText('{final_prompt}')">Copy to Clipboard</button>
"""))


Final Prompt After Adding Tags:





---



In [189]:
# Remove Tags from Danbooru Tags Cell
# @title ➖ Remove Tags from Danbooru Tags

# @markdown Check this box to reuse the result of this cell for the next run. It will not remove it from the input_tags visually though, but will do it in the background.
reuse_result = False # @param {type:"boolean"}
# @markdown This cell accepts a Danbooru tag list from the user and allows them to remove specific tags.

# @markdown Enter your Danbooru tag list here. Use to copy to clipboard from above cells if you want to use their outputs.
input_tags = "" # @param {type:"string"}

# @markdown Enter tags to remove from the existing tag list. Multiple tags can be separated by spaces.
tags_to_remove = "" # @param {type:"string"}

def remove_tags(input_tags, tags_to_remove):
    # Split input tags and tags to remove into lists
    base_tags = input_tags.split(' ')
    tags_to_remove = tags_to_remove.split(' ') if tags_to_remove else []

    # Remove only exact matches
    final_tags = [tag for tag in base_tags if tag not in tags_to_remove]

    # Create the final prompt
    final_prompt = ' '.join(final_tags)
    return final_prompt

# Get the final prompt based on user input
if reuse_result:
    input_tags = final_prompt
final_prompt = remove_tags(input_tags, tags_to_remove)

# Corrected lines for all relevant cells where '\n' was not properly formatted.

print("Final Prompt After Removing Tags:")
if only_cut_after_complete_tags:
    output_lines = []
    current_line = ""
    for tag in final_prompt.split(' '):
        if len(current_line) + len(tag) + 1 <= newline_threshold:
            current_line += (' ' if current_line else '') + tag
        else:
            output_lines.append(current_line)
            current_line = tag
    if current_line:
        output_lines.append(current_line)
    cleaned_output = '\n'.join(output_lines)  # Fixed to correctly use a properly formatted newline character
else:
    cleaned_output = '\n'.join([final_prompt[i:i + newline_threshold] for i in range(0, len(final_prompt), newline_threshold)])

print(cleaned_output)

display(HTML(f"""
    <button onclick="navigator.clipboard.writeText('{final_prompt}')">Copy to Clipboard</button>
"""))

Final Prompt After Removing Tags:





---



In [197]:
# Sort Tags Alphabetically Cell
# @title 🔤 Sort Tags Alphabetically
# @markdown This cell allows you to sort the tags alphabetically.

# @markdown Enter your Danbooru tag list here:
input_tags = ""  # @param {type:"string"}

def sort_tags_alphabetically(input_tags):
    # Split input tags into a list and sort alphabetically
    tags = input_tags.split(' ')
    sorted_tags = sorted(tags)

    # Create the final prompt
    final_prompt = ' '.join(sorted_tags)
    return final_prompt

# Get the final prompt based on user input
final_prompt = sort_tags_alphabetically(input_tags)

print("Final Prompt After Sorting Tags Alphabetically:")
if only_cut_after_complete_tags:
    output_lines = []
    current_line = ""
    for tag in final_prompt.split(' '):
        if len(current_line) + len(tag) + 1 <= newline_threshold:
            current_line += (' ' if current_line else '') + tag
        else:
            output_lines.append(current_line)
            current_line = tag
    if current_line:
        output_lines.append(current_line)
    cleaned_output = '\n'.join(output_lines)
else:
    cleaned_output = '\n'.join([final_prompt[i:i + newline_threshold] for i in range(0, len(final_prompt), newline_threshold)])

print(cleaned_output)

# Display a button (requires a suitable environment like Jupyter Notebook)
from IPython.core.display import HTML
display(HTML(f"""
    <button onclick="navigator.clipboard.writeText('{final_prompt}')">Copy to Clipboard</button>
"""))


Final Prompt After Sorting Tags Alphabetically:





---



In [198]:
# Clear Inputs/Outputs Cell
# @title ✨ Clear All Inputs and Outputs
# @markdown This cell resets all inputs and outputs to a clean state.

def clear_all():
    # Reset all global variables used across cells
    global input_prompt, cleaned_prompt, formatted_prompt, final_prompt, input_tags, additional_tags, tags_to_remove, reuse_result

    input_prompt = ""
    cleaned_prompt = ""
    formatted_prompt = ""
    final_prompt = ""
    input_tags = ""
    additional_tags = ""
    tags_to_remove = ""

    print("All inputs and outputs have been cleared.")

clear_all()

All inputs and outputs have been cleared.
